###  LLM Tour Recommendation

In [15]:
import torch
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_community.llms import Ollama
from langchain.schema import SystemMessage, HumanMessage
import json

# Enable MPS (M1 GPU) if available
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")


Using device: mps


In [16]:
# Initialize Ollama with the llama3.2 model
llm = Ollama(model="llama3.2")

In [17]:
# Define the JSON schema for the output
json_schema = {
    "type": "object",
    "properties": {
        "schedule": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "time": {"type": "string"},
                    "activity": {"type": "string"},
                    "duration": {"type": "string"}
                }
            }
        },
        "recommendations": {
            "type": "array",
            "items": {
                "type": "string"
            }
        }
    }
}

In [18]:

def generate_schedule_and_recommendations(date, temp_high, temp_low, weather, air_quality, attractions):
    prompt = f"""
    Generate a schedule and recommendations for a day in Busan on {date} with high temperature {temp_high}°C, low temperature {temp_low}°C, {weather} weather, {air_quality} air quality, and the following attractions: {attractions}.
    Provide a detailed schedule including recommended visit times, breaks, and any special considerations due to the weather and air quality.
    Also include recommendations for dealing with adverse conditions and optimizing the experience.
    
    Your response MUST be a valid JSON object with the following structure:
    {{
        "schedule": [
            {{"time": "HH:MM", "activity": "Description", "duration": "X hours"}}
        ],
        "recommendations": [
            "Recommendation 1",
            "Recommendation 2"
        ]
    }}
    """
    
    messages = [
        SystemMessage(content="You are a helpful assistant that generates travel schedules and recommendations in JSON format. But do not mention JSON in your response."),
        HumanMessage(content=prompt)
    ]
    
    response = llm.invoke(messages)
    print("Raw response:", response)  # Debug print
    try:
        return json.loads(response)
    except json.JSONDecodeError:
        # If JSON parsing fails, attempt to extract JSON from the response
        import re
        json_match = re.search(r'\{.*\}', response, re.DOTALL)
        if json_match:
            try:
                return json.loads(json_match.group())
            except json.JSONDecodeError:
                pass
        return {"error": "Failed to generate valid JSON", "raw_response": response}

In [19]:
# Data example for the itinerary
data = [
    {
        "date": "2024-10-05",
        "temp_high": 30,
        "temp_low": 14,
        "weather": "Windy",
        "air_quality": "Hazardous",
        "attractions": [
            {"name": "Haeundae Beach", "recommended_time": 2.98},
            {"name": "Gwangalli Beach", "recommended_time": 3.18},
            {"name": "Taejongdae", "recommended_time": 3.63},
            {"name": "Beomeosa Temple", "recommended_time": 2.4},
            {"name": "Gamcheon Culture Village", "recommended_time": 1.0}
        ]
    }
]

In [20]:
# Process each entry in the data
for entry in data:
    formatted_attractions = ", ".join([f"{attr['name']} ({attr['recommended_time']} hrs)" for attr in entry["attractions"]])
    result = generate_schedule_and_recommendations(
        entry["date"],
        entry["temp_high"],
        entry["temp_low"],
        entry["weather"],
        entry["air_quality"],
        formatted_attractions
    )
    print(f"Generated Output for {entry['date']}:")
    print(json.dumps(result, indent=2))

Raw response: {
    "schedule": [
        {
            "time": "08:00",
            "activity": "Start the day with a warm breakfast at a local café to prepare for the busy day ahead.",
            "duration": "1 hour"
        },
        {
            "time": "09:00",
            "activity": "Head to Beomeosa Temple (2.4 hours) and take in its serene atmosphere before the crowds arrive.",
            "duration": "2.4 hours"
        },
        {
            "time": "11:40",
            "activity": "Take a short break at a nearby park or café to enjoy a snack or drink while watching the weather forecast for any updates on air quality.",
            "duration": "0.8 hours"
        },
        {
            "time": "12:28",
            "activity": "Visit Gamcheon Culture Village (1 hour) and explore its colorful streets, alleys, and art galleries.",
            "duration": "1 hour"
        },
        {
            "time": "13:28",
            "activity": "Enjoy a packed lunch at one of the